<a href="https://colab.research.google.com/github/won-seop/CAPSTONE-DESIGN/blob/main/projectWeb_design(%EC%B5%9C%EC%A2%85).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install --upgrade openai

In [ ]:
!pip install flask
!pip install flask_ngrok
!pip install pyngrok==4.1.1
!pip install scikit-learn
!pip install joblib
!pip install tensorflow

In [ ]:
# // (ngrok 들어가서 개인 토큰 발급 받아서 사용할 것)
!ngrok authtoken  '' # 개인 ngrok authtoken 입력

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from flask import Flask, request, render_template
from flask_ngrok import run_with_ngrok
import math
import pickle
import pandas as pd
import os
template_folder = "/content/drive/MyDrive/Colab Notebooks/template"
static_folder = "/content/drive/MyDrive/Colab Notebooks/static"
app = Flask(__name__, template_folder=template_folder, static_url_path='/static', static_folder=static_folder)
run_with_ngrok(app)

In [ ]:
from flask import Flask, render_template, request
import openai
from openai import OpenAI
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
import joblib

client = OpenAI(api_key = '') # gpt key

# LSTM 모델과 토크나이저 로드
with open('/content/drive/MyDrive/Colab Notebooks/model/lstm.pickle', 'rb') as handle:
    loaded_tokenizer = pickle.load(handle)
model = load_model('/content/drive/MyDrive/Colab Notebooks/model/BiLstm.h5')

scaler = joblib.load('/content/drive/MyDrive/Colab Notebooks/model/scaler.pkl')
# 조회수 예측 함수
def predict_view(title):
    input_str = title
    input_df = pd.DataFrame(columns=['title', 'view'])
    input_df = input_df.append({'title' : input_str} , ignore_index=True)
    seq = loaded_tokenizer.texts_to_sequences(input_df['title'].values)
    padded = pad_sequences(seq, 47)
    pred = model.predict(padded)
    predicted_views = scaler.inverse_transform(pred).astype(int)[0][0]

    return predicted_views

def predict_top_title(titles):
    max_prediction = float('-inf')  # 최솟값으로 초기화
    top_title = None

    for title in titles:
        pred = predict_view(title)
        if pred > max_prediction:
            max_prediction = pred
            top_title = title

    return top_title, max_prediction

# OpenAI GPT 설정
model_id = "ft:gpt-3.5-turbo-0613:personal::85eQp6HM"

@app.route("/", methods=["GET", "POST"])
def my_index():
    top_title = ""
    generated_titles = []
    view_count_predictions = []
    max_prediction=[]

    if request.method == "POST":
        # 사용자 입력 처리 및 조회수 예측
        user_input = request.form.get("user_input")
        completions = []
        for _ in range(3):
            completion = client.chat.completions.create(
                model=model_id,
                messages=[
                    {"role": "system", "content": "문장이 말이되게끔 입력된 단어들이 들어간 유튜브 제목 생성해줘"},
                    {"role": "user", "content": user_input}
                ],
                max_tokens=70
            )
            completions.append(completion.choices[0].message.content)
        generated_titles = completions
        view_count_predictions = [predict_view(title) for title in generated_titles]
        top_title_index = view_count_predictions.index(max(view_count_predictions))
        top_title = generated_titles[top_title_index]

        display_output = True  # Set to True if you want to display the sections

        if display_output:
            return render_template("project3.html",data=zip(generated_titles, view_count_predictions),
                           top_title=top_title,display_output=True)

    return render_template("project3.html", data=zip(generated_titles, view_count_predictions),
                           top_title=top_title,display_output=False)


In [ ]:
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://9040-34-118-250-41.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [13/Nov/2023 14:22:55] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Nov/2023 14:22:56] "GET /static/projectStyle3.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Nov/2023 14:22:56] "GET /favicon.ico HTTP/1.1" 404 -
<ipython-input-10-f2510c65af09>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  input_df = input_df.append({'title' : input_str} , ignore_index=True)


1/1 [==============================] - 0s 24ms/step


...

INFO:werkzeug:127.0.0.1 - - [13/Nov/2023 14:25:36] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Nov/2023 14:25:36] "GET /static/projectStyle3.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [13/Nov/2023 14:25:44] "GET / HTTP/1.1" 200 -


1/1 [==============================] - 0s 25ms/step


<ipython-input-10-f2510c65af09>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  input_df = input_df.append({'title' : input_str} , ignore_index=True)
<ipython-input-10-f2510c65af09>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  input_df = input_df.append({'title' : input_str} , ignore_index=True)
<ipython-input-10-f2510c65af09>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  input_df = input_df.append({'title' : input_str} , ignore_index=True)


1/1 [==============================] - 0s 31ms/step


INFO:werkzeug:127.0.0.1 - - [13/Nov/2023 14:25:55] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Nov/2023 14:25:56] "GET /static/projectStyle3.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [13/Nov/2023 14:26:08] "GET / HTTP/1.1" 200 -
